## RMC Titanic Predicitive Analysis (73.9% Accuracy)

In [ ]:
import pandas as pd 
import numpy as np
from sklearn.impute import SimpleImputer

In [ ]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

In [ ]:
# train_data.head()
# test_data.head()

In [ ]:
# CHECK FOR COLUMNS WITH MISSING VALUES IN BOTH DATASET

dataset = [test_data, train_data]
for data in dataset:
    print(data.columns)

In [ ]:
# DROP CABIN COLUMN 
for data in dataset:
    data.drop(columns="Cabin", inplace=True)

# FILL NULL VALUES IN FARE COLUMN WITH THE AVERAGE 
for data in dataset:    
    if data["Fare"].isnull().any():
        data["Fare"] = data["Fare"].mean()
        
# DROP TICKET COLUMN 
for data in dataset:
    data.drop(columns="Ticket", inplace=True)
    
# FILL NULL VALUES IN EMBARKED COLUMN WITH "C"
for data in dataset:
    if data["Embarked"].isna().any():
        data["Embarked"] = "C"

In [ ]:
## FILL EMPTY AGE COLUMN WITH AGES BETWEEN THE MEAN AND STANDARD DEVIATION 

for data in dataset:
    
    # Calculate mean and standard deviation
    mean = data["Age"].mean()
    std = data["Age"].std()
    
    # Select columns where age isnull  
    empty_age = data["Age"].isna().sum()

    # Generate random numbers between mean and standard deviation 
    random_age = np.random.randint(mean-std, mean+std, size=empty_age)
    
    # Copy age column 
    age_slice = data["Age"].copy()
    
    # Let randomly generated numbers equal ages in copied column 
    age_slice[np.isnan(age_slice)] = random_age
    
    data["Age"] = age_slice
    data["Age"] = data["Age"].astype(int)

data["Age"].isna().sum()

In [ ]:
# Convert *all datatypes needed for predictive analysis to integers 

for data in dataset:
    #1
    data["Fare"] = data["Fare"].astype(int)
    #2
    gender = {"male": 0, "female": 1}
    data["Sex"] = data["Sex"].map(gender)

    #3
    ports = {"S": 1, "C": 2, "Q": 3}
    data["Embarked"] = data["Embarked"].map(ports)

In [ ]:
## MODEL BUILDING 

from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [ ]:
y_train = train_data["Survived"]
X_train = train_data[["Pclass", "Sex", "Age", "SibSp", "Parch", "Embarked"]]
X_test = test_data[["Pclass", "Sex", "Age", "SibSp", "Parch", "Embarked"]]


model = DecisionTreeRegressor()
model.fit(X_train,y_train)
test_data["Survived"] = model.predict(X_test)
test_data["Survived"] = test_data["Survived"].astype(int)


test = test_data[["PassengerId", "Survived"]]

In [ ]:
test.to_csv("final solution2.csv", index=False)